# 📊 SpectraMind V50 — Post-Submission Analysis & Leaderboard Tracking (Notebook 12)

**Goal.** Centralize post-submission metadata, track public leaderboard results, and maintain a lightweight, CI-friendly log of submissions and scores.

**What this notebook does**
1. Pre-flight (detect Kaggle CLI, set paths, read repo/git info)
2. Gather **local submission logs** (from prior notebooks) and consolidate into a historical CSV/JSON
3. (Optional) **Query Kaggle submissions** via CLI/API — DRY-RUN safe
4. Merge local logs with Kaggle metadata; compute **deltas** between runs and annotate best scoring submissions
5. Emit artifacts: `submissions_history.csv`, `best_submission.json`, and a **Mermaid** trend sketch


In [ ]:
# ░░ Pre-flight ░░
import os, sys, json, shutil, subprocess, datetime, pathlib, csv

RUN_TS = datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
RUN_ID = f"post_submit_{RUN_TS}"
ROOT_OUT = "/mnt/data/leaderboard_tracking"
ARTIFACTS = os.path.join(ROOT_OUT, RUN_ID)
LOGS = os.path.join(ARTIFACTS, "logs")
os.makedirs(ARTIFACTS, exist_ok=True); os.makedirs(LOGS, exist_ok=True)

def which(cmd:str)->bool: return shutil.which(cmd) is not None
KAGGLE_PRESENT = which("kaggle")

def git_cmd(args):
    try:
        out = subprocess.check_output(["git", *args], stderr=subprocess.STDOUT, timeout=5).decode().strip()
        return out
    except Exception:
        return None

env = {
    "python": sys.version.replace("\n"," "),
    "platform": sys.platform,
    "kaggle_present": KAGGLE_PRESENT,
    "run_id": RUN_ID,
    "paths": {"artifacts": ARTIFACTS, "logs": LOGS},
    "git": {
        "commit": git_cmd(["rev-parse", "HEAD"]),
        "branch": git_cmd(["rev-parse", "--abbrev-ref", "HEAD"]),
        "status": git_cmd(["status", "--porcelain"]),
    },
}
with open(os.path.join(ARTIFACTS, "env.json"), "w") as f:
    json.dump(env, f, indent=2)

print("=== Pre-flight ===")
print(json.dumps(env, indent=2))


## Consolidate local submission logs

In [ ]:
import glob, os, json, csv, hashlib

# Find prior local logs (from Notebook 11 or pipeline)
CANDIDATES = sorted(set(glob.glob("/mnt/data/**/submission_log.json", recursive=True)))
print("Found local logs:", len(CANDIDATES))

records = []
for path in CANDIDATES:
    try:
        data = json.load(open(path))
        # Normalize fields
        rec = {
            "source_path": path,
            "ts_utc": data.get("ts_utc") or data.get("timestamp_utc"),
            "run_id": data.get("run_id"),
            "submitted_file": data.get("submitted_file"),
            "kaggle_present": data.get("kaggle_present"),
            "submit_rc": data.get("submit_rc"),
        }
        # Attach file hash if available
        if rec["submitted_file"]:
            # try to resolve absolute path
            abs_guess = os.path.join(os.path.dirname(path), "..", "package", rec["submitted_file"])
            abs_guess = os.path.abspath(abs_guess)
            sha256 = None
            if os.path.exists(abs_guess):
                h = hashlib.sha256(); 
                with open(abs_guess, "rb") as f:
                    h.update(f.read())
                sha256 = h.hexdigest()
            rec["file_sha256"] = sha256
        records.append(rec)
    except Exception as e:
        print("Skip unreadable log:", path, e)

# Write a consolidated CSV snapshot for this run
snapshot_csv = os.path.join(ARTIFACTS, "submissions_snapshot.csv")
with open(snapshot_csv, "w", newline="") as f:
    w = csv.DictWriter(f, fieldnames=sorted({k for r in records for k in r.keys()}))
    w.writeheader()
    for r in records: w.writerow(r)

print("Snapshot rows:", len(records), "->", snapshot_csv)


## (Optional) Pull live Kaggle submission metadata — DRY-RUN safe

In [ ]:
import subprocess, json, os, shlex

COMPETITION = "ariel-data-challenge-2025"  # adjust if needed
def run_cmd(cmd_list, log_name):
    log_path = os.path.join(LOGS, f"{log_name}.log")
    err_path = os.path.join(LOGS, f"{log_name}.err")
    if not KAGGLE_PRESENT:
        msg = f"[DRY-RUN] Would execute: {' '.join(shlex.quote(c) for c in cmd_list)}\n"
        open(log_path, "w").write(msg); open(err_path, "w").write("")
        return 0, msg, ""
    try:
        proc = subprocess.Popen(cmd_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        out, err = proc.communicate()
        open(log_path, "wb").write(out or b""); open(err_path, "wb").write(err or b"")
        return proc.returncode, (out or b"").decode(), (err or b"").decode()
    except Exception as e:
        return 99, "", str(e)

# The Kaggle CLI can list submissions for a competition:
# kaggle competitions submissions -c <comp>
rc, out, err = run_cmd(["kaggle","competitions","submissions","-c", COMPETITION], "kaggle_submissions_list")
print("Submissions rc:", rc)
print("stdout (truncated):", out[:500])
print("stderr (truncated):", err[:500])

# Save raw listing for audit
open(os.path.join(ARTIFACTS, "kaggle_submissions_raw.txt"), "w").write(out if out else "")


## Parse Kaggle CLI table & merge

In [ ]:
import re, csv, io, json, os

kaggle_rows = []
table = open(os.path.join(ARTIFACTS, "kaggle_submissions_raw.txt")).read() if os.path.exists(os.path.join(ARTIFACTS, "kaggle_submissions_raw.txt")) else ""

# The Kaggle CLI prints a table; extract rows by splitting lines and using simple heuristics.
lines = [ln for ln in table.splitlines() if ln.strip()]
# Try to detect header separator line (----)
sep_idx = None
for i, ln in enumerate(lines):
    if set(ln.strip()) <= set("-|+ "):
        sep_idx = i
        break

header = []
if sep_idx is not None and sep_idx > 0:
    header_line = re.sub(r"\s+", " ", lines[sep_idx-1]).strip()
    header = header_line.split(" ")
    data_lines = lines[sep_idx+1:]
    for ln in data_lines:
        # Normalize whitespace columns (simple heuristic)
        parts = re.sub(r"\s{2,}", " | ", ln).split(" | ")
        if len(parts) >= len(header):
            row = dict(zip(header, parts[:len(header)]))
            kaggle_rows.append(row)

print("Parsed Kaggle rows:", len(kaggle_rows))

# Merge strategy: write a history CSV with both local logs and Kaggle columns (where available)
history_csv = os.path.join(ARTIFACTS, "submissions_history.csv")
all_fields = set()
for r in kaggle_rows: all_fields.update(r.keys())
for rec in records: all_fields.update(rec.keys())
all_fields = sorted(all_fields)

with open(history_csv, "w", newline="") as f:
    w = csv.DictWriter(f, fieldnames=all_fields)
    w.writeheader()
    # Prefer Kaggle rows (live) first, then append local records (with different fields)
    for r in kaggle_rows:
        w.writerow(r)
    for rec in records:
        w.writerow(rec)

print("Wrote consolidated history:", history_csv)


## Compute best submission & annotate deltas

In [ ]:
import csv, os, json

history_csv = os.path.join(ARTIFACTS, "submissions_history.csv")
best = None
rows = []
if os.path.exists(history_csv):
    with open(history_csv, newline="") as f:
        rdr = csv.DictReader(f)
        for r in rdr:
            rows.append(r)

    # Try to identify a PublicScore-like field (Kaggle CLI prints "PublicScore" for many comps)
    score_field = None
    for cand in ["PublicScore","Score","PublicScore*","Public_Score","publicScore"]:
        if rows and cand in rows[0]:
            score_field = cand; break

    # Convert scores to float if possible and sort desc
    def to_float(x):
        try: return float(x)
        except: return None

    scored = [(r, to_float(r.get(score_field))) for r in rows] if score_field else []
    scored = [(r, s) for (r, s) in scored if s is not None]
    if scored:
        scored.sort(key=lambda t: t[1], reverse=True)
        best = {"score_field": score_field, "row": scored[0][0], "score": scored[0][1]}

best_path = os.path.join(ARTIFACTS, "best_submission.json")
json.dump(best or {"note":"no scores found"}, open(best_path, "w"), indent=2)
print("Best submission summary ->", best_path)
print(json.dumps(best or {}, indent=2))


## Trend sketch (Mermaid)

> You can paste the following into your README to visualize simple submission flow.

```mermaid
flowchart TB
  A[Local submission logs] --> B[Consolidate snapshot]
  B --> C[Pull Kaggle submissions list]
  C --> D[Merge to history CSV]
  D --> E{Best score?}
  E -- yes --> F[Write best_submission.json]
  E -- no --> G[No score available]
```


## Browse produced artifacts

In [ ]:
import os

def tree(path, prefix=""):
    items = sorted(os.listdir(path))
    lines = []
    for i, name in enumerate(items):
        full = os.path.join(path, name)
        connector = "└── " if i == len(items)-1 else "├── "
        lines.append(prefix + connector + name)
        if os.path.isdir(full):
            extension = "    " if i == len(items)-1 else "│   "
            lines.extend(tree(full, prefix + extension))
    return lines

print("ARTIFACTS TREE:", ARTIFACTS)
print("\n".join(tree(ARTIFACTS)))


## Next steps
- Keep this notebook in CI after Notebook 11 to **log submissions automatically**.
- If the competition exposes a public submissions API/CSV, swap the CLI table parser for a JSON/CSV endpoint for better reliability.
- Extend the merge to include **config hash**, **data version** (from DVC), and Git tag for precise provenance.
- Consider a tiny dashboard (static HTML) that renders `submissions_history.csv` and highlights the best score per day.
